In [53]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

torch.__version__

'1.13.1+cu116'

In [10]:
NUM_EPOCHS = 5
nic = pd.read_csv('stevilka0.csv')
ena = pd.read_csv('stevilka1.csv')

nic.iloc[0]
data = pd.concat([nic, ena]) #združim


ndArray = data.to_numpy()
print(ndArray[0])


[ -99. 2639. 3181.  -19.  -80.  -41.   72. -166. -428.]


In [11]:
class AirWrite(nn.Module):
    def __init__(self):
        super(AirWrite, self).__init__()
        self.fc1 = nn.Linear(9, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

In [51]:
def train():
    train_loader = DataLoader(ndArray, batch_size = 5, shuffle = True)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = AirWrite().to(device)
    loss_fn = torch.nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)
    
    running_loss = 0.0
    total = 0
    correct = 0

    model.train()
    for epoch in range (NUM_EPOCHS):
        for i, data in enumerate(train_loader):

            inputs, targets = data
            inputs, targets = inputs.to(device), targets.to(device)
            
            optimizer.zero_grad()
            outputs = model(data)
            outputs = outputs.squeeze()

            loss = loss_fn(outputs, targets)
            loss.backward() 
            optimizer.step()  

            running_loss += loss.item() * data.size(0)
            predicted = outputs.round()
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

        epoch_loss = running_loss / total
        epoch_accuracy = correct / total
        print(f'Epoch {epoch+1}/{NUM_EPOCHS}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')
        print(device)

In [54]:
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)
    model = train()
    torch.save(model.state_dict(), "airWriting_model.pth")

cuda


ValueError: too many values to unpack (expected 2)